In [234]:
from random import random, choice, randint , choices
from pprint import pprint
from math import sin, cos
from expression_tree import Expression
from binarytree import Node , tree
import numpy as np
import graphviz
import sympy
import copy
# import logging

In [270]:
def f(x):
    return x**2 + 2*x

In [235]:
# logging.basicConfig(level=logging.DEBUG, file='../log/log1.txt',
#                     format='%(asctime)s  %(levelname)s : %(message)s')

*todo:*
- numpy type refactor
- overwriting variable x of fx with x = roulette
- sympy simplify
- numexpr
- deepcopy
- cut in mse same
- type portability of numpy.types
- jit numba

In [236]:
def operator_map(ch: str, left_sum, right_sum):
    # print(f"{ch=} ({type(ch).__name__}) | {left_sum=} ({type(left_sum).__name__}) | {right_sum=} ({type(right_sum).__name__})" )
    if ch == '+':
        return left_sum + right_sum
    elif ch == '-':
        return left_sum - right_sum
    elif ch == '*':
        return left_sum * right_sum
    elif ch == '/':
        # if right_sum == 0:
        #     return 0
        # TODO : bullshit
        return left_sum / right_sum
    elif ch == '^':
        return left_sum ** right_sum
    elif ch == 'cos':
        return cos(right_sum)
    elif ch == 'sin':
        return sin(right_sum)
    else:
        pass


In [237]:
def evaluate(root: Node, vars,dbg =0,depth =1 ):
    # if dbg == 0:
    #     print(root)
    #     print(f"{vars=}")
    # print(f"{root.value=} {depth=}")
    """evaluate value of tree from the input for variables using recursive inorder traverse"""
    if root is None: # empty tree
        return 0
    if root.left is None and root.right is None:  # leaf node
        if root.value.isnumeric():
            return int(root.value)
            # TODO : add check for is digit & in vars throw
        else:
            return vars
            
            # TODO : just int not float sin(1.45)
            # TODO : sin is radian
    left_sum = evaluate(root.left, vars,1,depth+1)
    right_sum = evaluate(root.right, vars,1,depth+1)
    return operator_map(root.value, left_sum, right_sum)

In [238]:
def expTree2str(root: Node):
    if root is None:  # empty tree
        return ""
    if root.left is None and root.right is None:  # leaf node
        return str(root.value)
        # TODO : add check for is digit & in vars throw

    left_sum = expTree2str(root.left)
    right_sum = expTree2str(root.right)
    return "(" + left_sum +str(root.value) +right_sum +")"



In [282]:
def MSE(actual, predictions):
    """Mean Squared Error"""
    sb = [(a1 - b1)**2 for (a1,b1) in zip(actual,predictions)]
    return sum(sb) / len(sb)

In [240]:
def crossover(x: Node, y:Node):
    # print("______________________________________________________")
    # print(x)
    # print(y)
    # print("++++++++++")
    x_copy = copy.deepcopy(x)
    y_copy = copy.deepcopy(y)
    inorder_x = x_copy.inorder
    inorder_y = y_copy.inorder
    # print(inorder_x, '\n',inorder_y)
    x_node = Node(1)
    y_node = Node(1)
    while x_node.value not in op:
        x_node = choice(inorder_x)
    while y_node.value not in op:
        y_node = choice(inorder_y)
    # print(x_node)
    # print(y_node)
    # print('res')
    x_node.__dict__, y_node.__dict__ = y_node.__dict__, x_node.__dict__
    # print(x_copy)
    # print(y_copy)
    return x_copy, y_copy


# (child1,child2) = crossover(first_tree, second_tree)


In [241]:
def roulette_wheel_selection(population, fit_vec: list):
    """Roulette Wheel Selection for minimizing problem"""
    maxfit = max(fit_vec)
    fitvec2 = [  (-1)*samp + maxfit for samp in fit_vec]
    return choices(population,weights= fitvec2, k= 1)[0]
    # TODO : negetive mse probability

In [242]:
UNARIES = ["sqrt(%s)", "exp(%s)", "log(%s)", "sin(%s)", "cos(%s)", "tan(%s)",
           "sinh(%s)", "cosh(%s)", "tanh(%s)", "asin(%s)", "acos(%s)",
           "atan(%s)", "-%s"]
BINARIES = ["%s + %s", "%s - %s", "%s * %s", "%s / %s", "%s ^ %s"]
BINARIES = ["%s + %s", "%s - %s", "%s * %s"]


PROP_PARANTHESIS = 0.4
PROP_BINARY = 1.0

In [243]:
def generate_expressions(scope, num_exp, num_ops):
    print(f"{num_ops=}")
    print(f"{num_exp=}")

    scope = list(scope)  # make a copy first, append as we go
    for _ in range(num_ops):
        if random() < PROP_BINARY:  # decide unary or binary operator
            ex = choice(BINARIES) % (choice(scope), choice(scope))
            if random() < PROP_PARANTHESIS:
                ex = "(%s)" % ex
            scope.append(ex)
        else:
            scope.append(choice(UNARIES) % choice(scope))
    return scope[-num_exp:]  # return most recent expressions

In [244]:
def main():
    # test()
    # np.set_printoptions(precision=3)

    x = list(range(1, 10))
    y = [f(samp) for samp in x]
    print(f"{x=}")
    print(f"{y=}")
    print(f"{scope=}")
    randexpr_arr = generate_expressions(scope, num_exp=5, num_ops=5)
    randexpr_exp = []
    randexpr_tree = []

    for index, expression in enumerate(randexpr_arr):
        test = Expression(expression=expression, operators=op, operators_info=op_info,
                          operators_associativity=assotiation, variables=varchar)
        randexpr_exp.append(test)
        randexpr_tree.append(test.tree())
        print(f"{index}:", randexpr_exp[index], "& ", expression)

        print(randexpr_tree[index])


    genetic_algo(population=randexpr_tree, mutation_probability=0)

In [246]:
op = {'+', '-', '*', '/', '^',
      'sin', 'cos'}
op_info = {'+': (2, 1), '-': (2, 1),
           '*': (2, 2), '/': (2, 2),
           '^': (2, 3),
           'sin': (1, 4), 'cos': (1, 4)}
assotiation = {'+': 'LR', '-': 'LR',
               '*': 'LR', '/': 'LR',
               '^': 'RL',
               'sin': 'RL', 'cos': 'RL'}
varchar = {'x'}
strscope = 'x'
scope = [c for c in strscope] 
# todo : expand scope


In [247]:
# def evaluate_vectorized(root: Node, vars: np.array):
#     if root is None: # empty tree
#         return 0
#     if root.left is None and root.right is None: # leaf node
#         return vars
#         # if root.value in scope:
#         #     return vars[]
#         #     # TODO : add check for is digit & in vars throw
#         # else:
#         #     return int(root.value)
#         #     # TODO : just int not float sin(1.45)
#         #     # TODO : sin is radian
#     left_sum = evaluate_vectorized(root.left, vars)
#     right_sum = evaluate_vectorized(root.right, vars)
#     return operator_map(root.value, left_sum, right_sum)

In [248]:
x = list(range(1, 10))
y = [f(samp) for samp in x]
# todo range from zero but / has problem

print(f"{x=}")
print(f"{y=}")
print(f"{scope=}")

x=[1, 2, 3, 4, 5, 6, 7, 8, 9]
y=[2, 6, 12, 20, 30, 42, 56, 72, 90]
scope=['x']


In [276]:
randexpr_arr = generate_expressions(scope, num_exp=10, num_ops=10)
randexpr_arr
simpled = [str(sympy.sympify(str(x))) for x in randexpr_arr]
# simpled = list({str(sympy.sympify(str(x))) for x in generate_expressions(scope, num_exp=10, num_ops=10)})
pprint(simpled)

num_ops=10
num_exp=10
['0',
 'x**2 - x',
 'x**3 - x',
 'x**3 - x',
 '-x',
 'x**3 - 4*x',
 '-2*x',
 '-x**4',
 'x*(x**3 - 4*x)',
 '-3*x']


In [266]:
randexpr_exp = []
randexpr_tree = []
y_pred = []
for index, expression in enumerate(randexpr_arr):
    test = Expression(expression=expression, operators=op, operators_info=op_info,
                      operators_associativity=assotiation, variables=varchar)
    randexpr_exp.append(test)
    randexpr_tree.append(test.tree())
    # print(f"{index}:", randexpr_exp[index], "& ", expression)
    # print(randexpr_tree[index])



In [252]:
def swapNode(x_node:Node,y_node:Node):
    x_node.__dict__, y_node.__dict__ = y_node.__dict__, x_node.__dict__

In [253]:
def evalFx(population:Node ,x:list ,y_pred):
    for index,tree in enumerate(population):
        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        # print(f"{index=}")
        # print("x: ", x)
        y_pred.append([evaluate(tree, te) for te in x])
    # y_pred= [[evaluate(tree,te) for te in x] for tree in population]


In [281]:
def genetic_algo(population: list, mutation_probability: float, saving: float, x: np.array, y: np.array):
    for abc in range(20):
        new_population = []
        y_pred = []
        evalFx(population ,x ,y_pred)
        mse = [MSE(y, gg) for gg in y_pred]

        population = [guy for _, guy in sorted(
            zip(mse, population), reverse=True, key=lambda x: x[0])]
        mse = sorted(mse, reverse=True)
        new_population.extend(population[:int(saving * len(population))])
        len_population = int(len(population) * (1-saving) / 2)
        for i in range(len_population):
            x_chosen = roulette_wheel_selection(population, mse)
            y_chosen = roulette_wheel_selection(population, mse)
            (child1, child2) = crossover(x_chosen, y_chosen)
            # if (random() < mutation_probability):
            #     mutate(child)
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
    y_pred = []
    evalFx(population ,x ,y_pred)
    mse = [MSE(y, gg) for gg in y_pred]
    population = [guy for _, guy in sorted(
                    zip(mse, population), reverse=True, key=lambda x: x[0])]
    mse = sorted(mse, reverse=True)
    return population


res = genetic_algo(population=randexpr_tree, saving=0.2,
                   mutation_probability=0.5, x=x, y=y)
print(res[0])
print(expTree2str(res[0]) )
print(str(
            sympy.sympify(
                str(expTree2str(res[0]))
            )
          )
      )
# for tree in res:
#     print(tree)
# simpled = [str(sympy.sympify(str(bruh))) for bruh in res]
# pprint(simpled)



                                                    __-__
                                                   /     \
    ______________________________________________*       +
   /                                               \     / \
  -______                                           x   x   x
 /       \
x       __+__
       /     \
      +       -______
     / \     /       \
    x   x   x       __-__________
                   /             \
                  +             __-______
                 / \           /         \
                x   x       __*         __-__
                           /   \       /     \
                          +     x     +       +
                         / \         / \     / \
                        x   x       x   x   x   x

(((x-((x+x)+(x-((x+x)-(((x+x)*x)-((x+x)-(x+x)))))))*x)-(x+x))
-2*x**3 - 2*x


In [110]:
# # strscope = "abcde"
# # scope = [c for c in strscope]
# # num_exp=randint(1,5),
# # randexpr = generate_expressions(scope,num_exp=int(input("enter num_exp: ")),num_ops=int(input("enter num_ops: ")))
# # for index,expression in enumerate(randexpr):
# #     print(expression)
# #     # print(f'{index=}------>',tree)
# # test = Expression(expression = "x+sin(90)^2*y",
# #                     operators = {'+', 'sin', '^', '*'},
# #                     operators_info = {'+': (2, 1), '*': (2, 2),'^': (2, 3), 'sin': (1, 4)},
# #                     operators_associativity = {'+': 'LR', '*': 'LR','^': 'RL', 'sin': 'RL'},
# #                     variables = {'x', 'y'})
# expr = "x+sin(90)^2*y"
# expr = '1+2*x^9/cos(x^2)+2*y^9'
# expr = '1+2*x^9/cos(x^2)^2+2*y^9'
# expr = "x+2*y^2"
# op = {'+', '-', '*', '/', '^',
#       'sin', 'cos'}
# op_info = {'+': (2, 1), '-': (2, 1),
#            '*': (2, 2), '/': (2, 2),
#            '^': (2, 3),
#            'sin': (1, 4), 'cos': (1, 4)}
# assotiation = {'+': 'LR', '-': 'LR',
#                '*': 'LR', '/': 'LR',
#                '^': 'RL',
#                'sin': 'RL', 'cos': 'RL'}
# varchar = {'x', 'y'}
# test = Expression(expression=expr, operators=op, operators_info=op_info,
#                   operators_associativity=assotiation, variables=varchar)
# print(expr)
# print(test.tree())
# test.tree().graphviz()
# # pprint(test._tokens)
# # pprint(list(expr))
# print(evaluate(test.tree(), {'x': 4, 'y': 2}))

In [57]:
# test.tree().graphviz()

In [141]:
first_tree = roulette_wheel_selection(randexpr_tree, mse)
second_tree = roulette_wheel_selection(randexpr_tree, mse)
print(first_tree)
print(second_tree)



        __+__
       /     \
    __*       *
   /   \     / \
  *     x   x   x
 / \
x   x


  *__________
 /           \
x           __+__
           /     \
        __*       *
       /   \     / \
      *     x   x   x
     / \
    x   x



In [143]:
(child1,child2) = crossover(first_tree, second_tree)

______________________________________________________

        __+__
       /     \
    __*       *
   /   \     / \
  *     x   x   x
 / \
x   x


  *__________
 /           \
x           __+__
           /     \
        __*       *
       /   \     / \
      *     x   x   x
     / \
    x   x

++++++++++

    __*
   /   \
  *     x
 / \
x   x


        __+__
       /     \
    __*       *
   /   \     / \
  *     x   x   x
 / \
x   x

res

            ______+__
           /         \
        __+__         *
       /     \       / \
    __*       *     x   x
   /   \     / \
  *     x   x   x
 / \
x   x


  *______
 /       \
x       __*
       /   \
      *     x
     / \
    x   x

